In [1]:
import cartopy.crs
from cartopy import feature
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean.cm
import numpy
import xarray
import pandas
import pathlib
import yaml

##### file paths and names

In [2]:
mesh_mask = xarray.open_dataset("https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn2DMeshMaskV17-02")
water_mask = mesh_mask.tmaskutil.isel(time=0)

fields = xarray.open_dataset("https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DTracerFields1hV19-05")
salinity = fields.salinity.sel(time="2020-08-14 14:30", depth=0, method="nearest").where(water_mask)

georef = xarray.open_dataset("https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetryV17-02")

location_dir = pathlib.Path('/Users/rmueller/Data/MIDOSS/AIS')
location_file = location_dir / 'Oil_Transfer_Facilities.xlsx'
wa_oil_in = pathlib.Path('/Users/rmueller/Data/MIDOSS/marine_transport_data') / 'WA_destination.yaml'
wa_oil_out = pathlib.Path('/Users/rmueller/Data/MIDOSS/marine_transport_data') / 'WA_origin.yaml'

In [3]:
vessel_types = [
    'tanker', 
    'atb', 
    'barge'
]
oil_types = [
    'akns', 
    'bunker', 
    'dilbit', 
    'jet', 
    'diesel', 
    'gas', 
    'other'
]
oil_colors = [
    'darkolivegreen',
    'olivedrab',
    'slategrey',
    'indigo',
    'mediumslateblue',
    'cornflowerblue',
    'saddlebrown'
]
gallons_to_liters = 3.78541

### Load locations of marine transfer facilities

##### read in oil attribute data

In [4]:
with open(wa_oil_in) as file:
        oil_attrs_in = yaml.load(file, Loader=yaml.Loader)
        
with open(wa_oil_out) as file:
        oil_attrs_out = yaml.load(file, Loader=yaml.Loader)

In [5]:
oil_attrs_in['Alon Asphalt Company (Paramount Petroleum)']['barge']['diesel']

{'fraction_of_total': 0.6440647319410542,
 'number_of_transfers': 28,
 'total_gallons': 14960400}

In [6]:
oil_attrs_in.keys()

dict_keys(['Alon Asphalt Company (Paramount Petroleum)', 'Andeavor Anacortes Refinery (formerly Tesoro)', 'BP Cherry Point Refinery', 'Kinder Morgan Liquids Terminal - Harbor Island', 'Maxum (Rainer Petroleum)', 'NAVSUP Manchester', 'Naval Air Station Whidbey Island (NASWI)', 'Nustar Energy Tacoma', 'Phillips 66 Ferndale Refinery', 'Phillips 66 Tacoma Terminal', 'REG Grays Harbor, LLC', 'SeaPort Sound Terminal', 'Shell Oil LP Seattle Distribution Terminal', 'Shell Puget Sound Refinery', 'TLP Management Services LLC (TMS)', 'Tesoro Pasco Terminal', 'Tesoro Port Angeles Terminal', 'Tesoro Vancouver Terminal', 'Tidewater Snake River Terminal', 'Tidewater Vancouver Terminal', 'U.S. Oil & Refining', 'names'])

##### read in locations for marine transfer facilities

In [7]:
wa_locs = pandas.read_excel(location_file, sheet_name='Washington', 
                   usecols="B,I,J")
bc_locs = pandas.read_excel(location_file, sheet_name='British Columbia', 
                   usecols="A,B,C")

##### find and correct entries that differ between DOE naming and Casey's name list (I plan to correct these names so we are self-consistent)

In [8]:
for location in wa_locs['FacilityName']:
    if location not in oil_attrs_in.keys():
        print(location)

Maxum Petroleum - Harbor Island Terminal
Marathon Anacortes Refinery (formerly Tesoro)
Nustar Energy Vancouver


In [9]:
# Maxum Petroleum - Harbor Island Terminal -> 'Maxum (Rainer Petroleum)'
# Marathon Anacortes Refinery (formerly Tesoro) -> 'Andeavor Anacortes Refinery (formerly Tesoro)'
# Nustar Energy Vancouver -> Not included

In [10]:
for location in wa_locs['FacilityName']:
    if location == 'Marathon Anacortes Refinery (formerly Tesoro)':
#         wa_locs[wa_locs['FacilityName']=='Marathon Anacortes Refinery (formerly Tesoro)'].FacilityName = (
#             'Andeavor Anacortes Refinery (formerly Tesoro)'
#         )
         wa_locs.loc[wa_locs['FacilityName']=='Marathon Anacortes Refinery (formerly Tesoro)','FacilityName'] = (
                    'Andeavor Anacortes Refinery (formerly Tesoro)'
                )
    elif location == 'Maxum Petroleum - Harbor Island Terminal':
        wa_locs.loc[wa_locs['FacilityName']=='Maxum Petroleum - Harbor Island Terminal','FacilityName'] = (
            'Maxum (Rainer Petroleum)'
        )
wa_locs.drop(index=17, inplace=True)
wa_locs.reset_index( drop=True, inplace=True )
wa_locs

,FacilityName,DockLatNumber,DockLongNumber
0,BP Cherry Point Refinery,48.86111,-122.758
1,Shell Puget Sound Refinery,48.50945,-122.577
2,Shell Oil LP Seattle Distribution Terminal,47.58870,-122.353
3,Maxum (Rainer Petroleum),47.58753,-122.353
4,Tidewater Snake River Terminal,46.22312,-119.014
5,Nustar Energy Tacoma,47.26091,-122.436
6,SeaPort Sound Terminal,47.27679,-122.389
7,Tesoro Vancouver Terminal,45.63551,-122.703
8,Phillips 66 Ferndale Refinery,48.82600,-122.720
9,Phillips 66 Tacoma Terminal,47.25816,-122.434


In [11]:
# remove locations outside of Salish Sea
non_salish_locs = wa_locs[wa_locs['DockLatNumber']<47].index
wa_locs.drop(index=non_salish_locs, inplace=True)
wa_locs.reset_index( drop=True, inplace=True )
wa_locs

,FacilityName,DockLatNumber,DockLongNumber
0,BP Cherry Point Refinery,48.86111,-122.758
1,Shell Puget Sound Refinery,48.50945,-122.577
2,Shell Oil LP Seattle Distribution Terminal,47.58870,-122.353
3,Maxum (Rainer Petroleum),47.58753,-122.353
4,Nustar Energy Tacoma,47.26091,-122.436
5,SeaPort Sound Terminal,47.27679,-122.389
6,Phillips 66 Ferndale Refinery,48.82600,-122.720
7,Phillips 66 Tacoma Terminal,47.25816,-122.434
8,Andeavor Anacortes Refinery (formerly Tesoro),48.50837,-122.569
9,Tesoro Port Angeles Terminal,48.13615,-123.462


### Add oil entries to dataframe

In [12]:
for oil in oil_types:
    wa_locs[f'{oil}_in'] = numpy.zeros(len(wa_locs['FacilityName']))
    wa_locs[f'{oil}_in_scale'] = numpy.zeros(len(wa_locs['FacilityName']))
    
    wa_locs[f'{oil}_out'] = numpy.zeros(len(wa_locs['FacilityName']))
    wa_locs[f'{oil}_out_scale'] = numpy.zeros(len(wa_locs['FacilityName']))

# Sum volume of all oil types
wa_locs['all_in'] = numpy.zeros(len(wa_locs['FacilityName']))
wa_locs['all_out'] = numpy.zeros(len(wa_locs['FacilityName']))

### loop through all facilities and get the total amount of fuel by fuel type

In [13]:
for index in range(len(wa_locs)):
    for oil in oil_types:
        for vessel in vessel_types:
            # add oil accross vessel types and convert to liters while we are at it
            wa_locs.loc[index, f'{oil}_in'] += oil_attrs_in[wa_locs.FacilityName[index]][vessel][oil]['total_gallons']
            wa_locs.loc[index, f'{oil}_out'] += oil_attrs_out[wa_locs.FacilityName[index]][vessel][oil]['total_gallons']
        
        # add oil across oil types
        wa_locs.loc[index,'all_in'] += wa_locs.loc[index,f'{oil}_in']
        wa_locs.loc[index,'all_out'] += wa_locs.loc[index,f'{oil}_out']

# Now make a scale to use for marker size
for oil in oil_types:
    wa_locs[f'{oil}_in_scale'] = wa_locs[f'{oil}_in']/wa_locs[f'{oil}_in'].sum()
    wa_locs[f'{oil}_out_scale'] = wa_locs[f'{oil}_out']/wa_locs[f'{oil}_out'].sum()
        
wa_locs['all_in_scale'] = wa_locs['all_in']/wa_locs['all_in'].sum()
wa_locs['all_out_scale'] = wa_locs['all_out']/wa_locs['all_out'].sum()

In [14]:
wa_locs

,FacilityName,DockLatNumber,DockLongNumber,akns_in,akns_in_scale,akns_out,akns_out_scale,bunker_in,bunker_in_scale,bunker_out,...,gas_out,gas_out_scale,other_in,other_in_scale,other_out,other_out_scale,all_in,all_out,all_in_scale,all_out_scale
0,BP Cherry Point Refinery,48.86111,-122.758,1.652454e+09,0.429764,18480000.0,0.103413,74351256.0,0.186133,0.0,...,558038880.0,0.413752,133726320.0,0.423276,126882000.0,0.304566,1.995820e+09,1.435339e+09,0.385913,0.338572
1,Shell Puget Sound Refinery,48.50945,-122.577,1.264963e+09,0.328987,25200000.0,0.141018,281820.0,0.000706,88452000.0,...,301035000.0,0.223199,43691750.0,0.138295,31395000.0,0.075360,1.319534e+09,6.584760e+08,0.255146,0.155323
2,Shell Oil LP Seattle Distribution Terminal,47.58870,-122.353,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,4.242000e+06,0.000000e+00,0.000820,0.000000
3,Maxum (Rainer Petroleum),47.58753,-122.353,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,115506.0,0.000277,2.800000e+04,8.395546e+06,0.000005,0.001980
4,Nustar Energy Tacoma,47.26091,-122.436,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,200000.0,0.000480,3.360000e+06,7.670000e+05,0.000650,0.000181
5,SeaPort Sound Terminal,47.27679,-122.389,0.000000e+00,0.000000,0.0,0.000000,92118848.0,0.230613,150318766.0,...,0.0,0.000000,11314019.0,0.035812,39434132.0,0.094657,1.751479e+08,1.966947e+08,0.033867,0.046397
6,Phillips 66 Ferndale Refinery,48.82600,-122.720,5.674200e+08,0.147572,0.0,0.000000,924000.0,0.002313,204510600.0,...,276444000.0,0.204967,7518000.0,0.023796,30177000.0,0.072437,5.775084e+08,6.744906e+08,0.111667,0.159101
7,Phillips 66 Tacoma Terminal,47.25816,-122.434,0.000000e+00,0.000000,0.0,0.000000,3423000.0,0.008569,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,9.419180e+07,0.000000e+00,0.018213,0.000000
8,Andeavor Anacortes Refinery (formerly Tesoro),48.50837,-122.569,3.402405e+08,0.088488,96024000.0,0.537345,8400000.0,0.021029,159262194.0,...,201096000.0,0.149101,91988400.0,0.291165,85501800.0,0.205238,5.716697e+08,7.017696e+08,0.110538,0.165535
9,Tesoro Port Angeles Terminal,48.13615,-123.462,0.000000e+00,0.000000,0.0,0.000000,21151996.0,0.052953,25769625.0,...,0.0,0.000000,315000.0,0.000997,1356600.0,0.003256,2.424521e+07,2.954379e+07,0.004688,0.006969


### Sort facilities into latitude bins

In [15]:
lat_partition = [48, 48.15, 48.3, 48.7]

south  = wa_locs[wa_locs.DockLatNumber < lat_partition[0]]

portangeles = wa_locs[
    (wa_locs.DockLatNumber > lat_partition[0]) & 
    (wa_locs.DockLatNumber < lat_partition[1])
]

naswi = wa_locs[
    (wa_locs.DockLatNumber > lat_partition[1]) & 
    (wa_locs.DockLatNumber < lat_partition[2])
]

anacortes  = wa_locs[
    (wa_locs.DockLatNumber > lat_partition[2]) & 
    (wa_locs.DockLatNumber < lat_partition[3])
]

north = wa_locs[wa_locs.DockLatNumber > lat_partition[3]]

# sort indices to make life easy when plotting up
south.reset_index( drop=True, inplace=True )
portangeles.reset_index( drop=True, inplace=True )
naswi.reset_index( drop=True, inplace=True )
anacortes.reset_index( drop=True, inplace=True )
north.reset_index( drop=True, inplace=True )

In [24]:
# create vectors of oil volumes by oil type for graphs
south_oils = {}
south_oils['import'] = numpy.zeros(len(oil_types))
south_oils['export'] = numpy.zeros(len(oil_types))

portangeles_oils = {}
portangeles_oils['import'] = numpy.zeros(len(oil_types))
portangeles_oils['export'] = numpy.zeros(len(oil_types))

naswi_oils = {}
naswi_oils['import'] = numpy.zeros(len(oil_types))
naswi_oils['export'] = numpy.zeros(len(oil_types)) 

anacortes_oils = {}
anacortes_oils['import'] = numpy.zeros(len(oil_types))
anacortes_oils['export'] = numpy.zeros(len(oil_types)) 

north_oils = {}
north_oils['import'] = numpy.zeros(len(oil_types))
north_oils['export'] = numpy.zeros(len(oil_types))

for index in range(len(oil_types)):
    south_oils['import'][index]       = south[f'{oil_types[index]}_in'].sum() * gallons_to_liters 
    portangeles_oils['import'][index] = portangeles[f'{oil_types[index]}_in'].sum() * gallons_to_liters 
    naswi_oils['import'][index]       = naswi[f'{oil_types[index]}_in'].sum() * gallons_to_liters
    anacortes_oils['import'][index]   = anacortes[f'{oil_types[index]}_in'].sum() * gallons_to_liters 
    north_oils['import'][index]       = north[f'{oil_types[index]}_in'].sum() * gallons_to_liters 
    
    south_oils['export'][index]       = south[f'{oil_types[index]}_out'].sum() * gallons_to_liters 
    portangeles_oils['export'][index] = portangeles[f'{oil_types[index]}_out'].sum() * gallons_to_liters 
    naswi_oils['export'][index]       = naswi[f'{oil_types[index]}_out'].sum() * gallons_to_liters
    anacortes_oils['export'][index]   = anacortes[f'{oil_types[index]}_out'].sum() * gallons_to_liters 
    north_oils['export'][index]       = north[f'{oil_types[index]}_out'].sum() * gallons_to_liters 

### Plot by facility groups (dipping toes in)

In [ ]:
# the offsets needed for indexing from lower left of graphic boundary
x0 = -0.12
y0 = -0.044

# mapping specifications
rotated_crs = cartopy.crs.RotatedPole(pole_longitude=120.0, pole_latitude=63.75)
plain_crs = cartopy.crs.PlateCarree()

fig,ax = plt.subplots(
    1, 1, figsize=(12, 12), subplot_kw={"projection": rotated_crs, "facecolor": "#8b7765"}
)
ax.add_feature(feature.GSHHSFeature('full', edgecolor='k', facecolor='burlywood'))
quad_mesh = ax.pcolormesh(
    georef.longitude, georef.latitude, salinity, transform=plain_crs, cmap=cmocean.cm.haline, shading="auto"
)

# calculate location of marine terminal within graphic window

#~~~~~~~~~~~~~~~~ NORTH ~~~~~~~~~~~~~~~~~~
xleft = ((north.DockLongNumber[0] - georef.longitude.min())/(georef.longitude.max() - georef.longitude.min())).values.item()
ybottom   = ((north.DockLatNumber[0] - georef.latitude.min())/(georef.latitude.max() - georef.latitude.min())).values.item()

ax_north = fig.add_axes([xleft+x0, ybottom+1.7*y0,.08, .2])
ax_north.pie(north_oils['import'], colors = oil_colors)

#~~~~~~~~~~~~~~~~ SOUTH ~~~~~~~~~~~~~~~~~~
xleft = ((south.DockLongNumber[0] - georef.longitude.min())/(georef.longitude.max() - georef.longitude.min())).values.item()
ybottom   = ((south.DockLatNumber[0] - georef.latitude.min())/(georef.latitude.max() - georef.latitude.min())).values.item()

ax_south = fig.add_axes([xleft+2*x0, ybottom+y0,.08, .2])
ax_south.pie(south_oils['import'], colors = oil_colors)

#~~~~~~~~~~~~~~~~ ANACORTES ~~~~~~~~~~~~~~~~~~
xleft = ((anacortes.DockLongNumber[0] - georef.longitude.min())/(georef.longitude.max() - georef.longitude.min())).values.item()
ybottom   = ((anacortes.DockLatNumber[0] - georef.latitude.min())/(georef.latitude.max() - georef.latitude.min())).values.item()

ax_anacortes = fig.add_axes([xleft+1.2*x0, ybottom+2.7*y0,.08, .2])
ax_anacortes.pie(anacortes_oils['import'], colors = oil_colors)

#~~~~~~~~~~~~~~~~ PORT ANGELES ~~~~~~~~~~~~~~~~~~
xleft = ((portangeles.DockLongNumber[0] - georef.longitude.min())/(georef.longitude.max() - georef.longitude.min())).values.item()
ybottom   = ((portangeles.DockLatNumber[0] - georef.latitude.min())/(georef.latitude.max() - georef.latitude.min())).values.item()

ax_portangeles = fig.add_axes([xleft+x0, ybottom+2.3*y0,.08, .2])
ax_portangeles.pie(portangeles_oils['import'], colors = oil_colors)


#ax_north.bar(oil_types, 1e1* numpy.ones(7), width = 1)
plt.show()
fig.canvas.draw()
fig.tight_layout()
    

###### 